In [1]:
import cv2
import numpy as np

In [2]:
# 取得する色の範囲を指定する 
lower = np.array([20, 20, 20])
upper = np.array([140, 140, 140])

# 閾値
THRES = 50

#　黒板の数
num_blackboard = 6

In [4]:
def concat_rect(rect_dict, rect_S_dict, num_blackboard):
    """
    :param rect_dict: dict
    :param rect_S_dict: dict
    :param num_blackboard: int
    :return: int*4
    """

    counter = 0
    minx = miny = 1000000
    max_x = max_y = 0
    for k, v in sorted(rect_S_dict.items(), key=lambda x: -x[1]):
        x = rect_dict[k][0]
        y = rect_dict[k][1]
        w = rect_dict[k][2]
        h = rect_dict[k][3]

        if minx >= x:
            minx = x
        if miny >= y:
            miny = y
        if max_x <= x + w:
            max_x = x + w
        if max_y <= y + h:
            max_y = y + h

        counter += 1
        if counter >= num_blackboard:
            return minx, miny, max_x, max_y

# load image, change color spaces, and smoothing
img = cv2.imread('../../project_ensyu/input/blackboard.png')
img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# img_HSV = cv2.GaussianBlur(img_HSV, (9, 9), 3)

# detect tulips
img_H, img_S, img_V = cv2.split(img_HSV)

# 指定した色に基づいたマスク画像の生成
img_mask = cv2.inRange(img_HSV, lower, upper)

# フレーム画像とマスク画像の共通の領域を抽出する。
img_color = cv2.bitwise_and(img, img, mask=img_mask)

# gray変換
gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
cv2.imwrite('output/blackboard_gray.jpg', gray)
    
# 閾値    
_thre, img_th = cv2.threshold(gray, THRES, 255, cv2.THRESH_BINARY)
cv2.imwrite('output/blackboard_th.jpg', img_th)

# find tulips
labels, contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

rect_S_dict = {}
rect_dict = {}

for i in range(0, len(contours)):
    if len(contours[i]) > 0:

        # remove small objects
        if cv2.contourArea(contours[i]) < 800:
            continue

        rect = contours[i]
        x, y, w, h = cv2.boundingRect(rect)
        
        rect_dict[i] = [x,y,w,h]
        S = w*h
        rect_S_dict[i] = S

# dictの降順ソート
minx, miny, max_x, max_y = concat_rect(rect_dict, rect_S_dict, num_blackboard)
cv2.rectangle(img, (minx, miny), (max_x, max_y), (0, 0, 255), 10)
    
# save
cv2.imwrite('output/blackboard_obs.jpg', img)

True

In [14]:
cap = cv2.VideoCapture('input/black_board.mov')

while(1):
    
    ret, img = cap.read()

    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # img_HSV = cv2.GaussianBlur(img_HSV, (9, 9), 3)

    # detect tulips
    img_H, img_S, img_V = cv2.split(img_HSV)

    # 指定した色に基づいたマスク画像の生成
    img_mask = cv2.inRange(img_HSV, lower, upper)

    # フレーム画像とマスク画像の共通の領域を抽出する。
    img_color = cv2.bitwise_and(img, img, mask=img_mask)

    # gray変換
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # 閾値    
    _thre, img_th = cv2.threshold(gray, THRES, 255, cv2.THRESH_BINARY)

    # find tulips
    labels, contours, hierarchy = cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    rect_S_dict = {}
    rect_dict = {}

    for i in range(0, len(contours)):
        if len(contours[i]) > 0:

            # remove small objects
            if cv2.contourArea(contours[i]) < 1000:
                continue

            rect = contours[i]
            x, y, w, h = cv2.boundingRect(rect)

            rect_dict[i] = [x,y,w,h]
            S = w*h
            rect_S_dict[i] = S

    # dictの降順ソート
    counter = 0
    for k, v in sorted(rect_S_dict.items(), key=lambda x: -x[1]):
        x = rect_dict[k][0]
        y = rect_dict[k][1]  
        w = rect_dict[k][2]
        h = rect_dict[k][3]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 10)
        counter += 1
        if counter >= num_blackboard:
            break
    
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
 
    # qを押したら終了
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
counter = 0
minx = 100000
miny = 100000
max_x = 0
max_y = 0

for k, v in sorted(rect_S_dict.items(), key=lambda x: -x[1]):
    x = rect_dict[k][0]
    y = rect_dict[k][1]  
    w = rect_dict[k][2]
    h = rect_dict[k][3]
    
    if minx >= x:
        minx = x
    if miny >= y-h:
        miny = y-h
    if max_x <= x+w:
        max_x = x+w
    if max_y <= y:
        max_y = y
    
    counter += 1
    if counter >= num_blackboard:
        break